
<br>
Example of running GROMACS with lithops<br>


In [1]:
import lithops
import os
import zipfile
import time
import wget
import json

In [2]:
temp_dir = '/tmp'
iterdata = [1]

In [3]:
def sh_cmd_executor(x, param1, ibm_cos):
    lithops_config = json.loads(os.environ['LITHOPS_CONFIG'])
    bucket = lithops_config['lithops']['storage_bucket']
    print (bucket)
    print (param1)
    filename = 'benchMEM.zip'
    outfile = os.path.join(temp_dir, filename)
    if not os.path.isfile(filename):
        filename = wget.download('https://www.mpibpc.mpg.de/15101317/benchMEM.zip', out=outfile)
        print(filename, "was downloaded")
        with zipfile.ZipFile(outfile, 'r') as zip_ref:
            print('Extracting file to %s' % temp_dir)
            zip_ref.extractall(temp_dir)
    else:
        print(filename, " already exists")
    os.chdir(temp_dir)
    cmd = "/usr/local/gromacs/bin/gmx mdrun -nt 4 -s benchMEM.tpr -nsteps 1000 -resethway"
    st = time.time()
    import subprocess
    subprocess.call(cmd, shell=True)
    run_time = time.time() - st

    # upload results to IBM COS
    res = ['confout.gro', 'ener.edr', 'md.log',  'state.cpt']
    for name in res:
        f = open(os.path.join(temp_dir, name), "rb")
        ibm_cos.put_object(Bucket=bucket, Key=os.path.join('gmx-mem', name), Body=f)
    with open('md.log', 'r') as file:
        data = file.read()
    return {'run_time': run_time, 'md_log': data}

In [4]:
if __name__ == '__main__':
    # Example of using bechMEM from https://www.mpibpc.mpg.de/grubmueller/bench
    param1 = 'param1 example'
    total_start = time.time()
    fexec = lithops.FunctionExecutor(runtime='cactusone/lithops-gromacs:1.0.2', runtime_memory=2048)
    fexec.map(sh_cmd_executor, iterdata, extra_args=(param1,))
    res = fexec.get_result()
    fexec.clean()
    print ("GROMACS execution time {}".format(res[0]['run_time']))
    print ("Total execution time {}".format(time.time()-total_start))
    print (res[0]['md_log'])

2021-09-18 21:41:12,599 [INFO] lithops.config -- Lithops v2.5.0
2021-09-18 21:41:12,728 [INFO] lithops.storage.backends.ibm_cos.ibm_cos -- IBM COS Storage client created - Region: eu-de
2021-09-18 21:41:14,111 [INFO] lithops.serverless.backends.ibm_cf.ibm_cf -- IBM CF client created - Region: eu-de - Namespace: ORGece1_ecespace1
2021-09-18 21:41:14,113 [INFO] lithops.invokers -- ExecutorID 31b5be-0 | JobID M000 - Selected Runtime: ecenurarslan/gromacs-runtime:0.1 - 2048MB
2021-09-18 21:41:14,559 [INFO] lithops.invokers -- ExecutorID 31b5be-0 | JobID M000 - Starting function invocation: sh_cmd_executor() - Total: 1 activations
2021-09-18 21:41:14,560 [INFO] lithops.invokers -- ExecutorID 31b5be-0 | JobID M000 - View execution logs at /tmp/lithops/logs/31b5be-0-M000.log
2021-09-18 21:41:14,561 [INFO] lithops.wait -- ExecutorID 31b5be-0 - Getting results from functions


    0%|          | 0/1  

2021-09-18 21:42:10,790 [INFO] lithops.executors -- ExecutorID 31b5be-0 - Cleaning temporary data


GROMACS execution time 53.15668082237244
Total execution time 58.22850155830383
                      :-) GROMACS - gmx mdrun, 2020.5 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf 